In [114]:
# This notebook demonstrates how to subscribe to a KE and perform an ASK KI

In [115]:
# Defining constants

# Knowledge engine REST API URL
URL = "http://127.0.0.1:8280/rest"

# Knowledge Base ID
KB_ID = "http://127.0.0.1:8000/subscriber"

# Knowledge engine REST API headers
HEADERS = {
    'Content-Type': 'application/json',
    'Knowledge-Base-Id': KB_ID,
}

In [116]:

# register API in the knowledge engine
import httpx

# Creating a smart connector
def create_smart_connector():
    # Smart connector data
    sc_data = {
        "knowledgeBaseId": KB_ID,
        "knowledgeBaseName": "Subscriber",
        "knowledgeBaseDescription": "Subscriber",
        "reasonerEnabled": "false",
    }

    # Registering the smart connector via the /sc endpoint
    response = httpx.post(URL + "/sc", headers=HEADERS, json=sc_data)

    print(response.status_code)
    print(response.text)

create_smart_connector()


200



In [117]:

# check if the smart connector is registered
def check_smart_connector():
    response = httpx.get(URL + "/sc", headers=HEADERS)
    print(response.status_code)
    print(response.text)

check_smart_connector()


200
[{"knowledgeBaseId":"http://127.0.0.1:8000/subscriber","knowledgeBaseName":"Subscriber","knowledgeBaseDescription":"Subscriber","reasonerEnabled":false}]


In [118]:
# register an ASK Knowledge Interaction with the smart connector
def register_ask_ki():
    ask_ki_data = {
        "knowledgeInteractionType": "AskKnowledgeInteraction",
        "prefixes": {"rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
                     "saref": "https://w3id.org/saref#",
                     },

        "graphPattern": '''?meas rdf:type saref:Measurement .
                            ?meas saref:hasValue ?temp .
                            ?meas saref:isMeasuredIn saref:TemperatureUnit .
                            ?meas saref:hasTimestamp ?timestamp .
                            ?meas saref:isMeasurementOf ?room_id .
                            ?meas saref:relatesToProperty saref:Temperature .
                            ?meas saref:measurementMadeBy ?device_id .'''
    }

    response = httpx.post(URL + "/sc/ki", headers=HEADERS, json=ask_ki_data)
    print(response.status_code)
    print(response.text)

register_ask_ki()


200
{"knowledgeInteractionId":"http://127.0.0.1:8000/subscriber/interaction/977da25d-3c10-457b-8d48-3876e34b7d1e"}


In [119]:
# check if the Knowledge Interaction is registered
def list_all_kis():
    response = httpx.get(URL + "/sc/ki", headers=HEADERS)
    return response.json()
list_all_kis()



[{'knowledgeInteractionId': 'http://127.0.0.1:8000/subscriber/interaction/977da25d-3c10-457b-8d48-3876e34b7d1e',
  'knowledgeInteractionType': 'AskKnowledgeInteraction',
  'communicativeAct': {'requiredPurposes': ['https://w3id.org/knowledge-engine/InformPurpose'],
   'satisfiedPurposes': ['https://w3id.org/knowledge-engine/InformPurpose']},
  'graphPattern': '?meas <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://w3id.org/saref#Measurement> . ?meas <https://w3id.org/saref#hasValue> ?temp . ?meas <https://w3id.org/saref#isMeasuredIn> <https://w3id.org/saref#TemperatureUnit> . ?meas <https://w3id.org/saref#hasTimestamp> ?timestamp . ?meas <https://w3id.org/saref#isMeasurementOf> ?room_id . ?meas <https://w3id.org/saref#relatesToProperty> <https://w3id.org/saref#Temperature> . ?meas <https://w3id.org/saref#measurementMadeBy> ?device_id . '}]

In [120]:
list_all_kis()[-1]['knowledgeInteractionId']

'http://127.0.0.1:8000/subscriber/interaction/977da25d-3c10-457b-8d48-3876e34b7d1e'

In [121]:
# Asking the question
def ask_ki():
    ki_id = list_all_kis()[-1]['knowledgeInteractionId']
    ask_headers = {
        #'Knowledge-Base-Id': "http://127.0.0.1:8000/thermostat",
        'Knowledge-Base-Id': KB_ID,
        'Knowledge-Interaction-Id': ki_id,
    }

    ask_data = [{ "device_id": "http://127.0.0.1:8000/thermostat"}]

    response = httpx.post(URL + "/sc/ask", headers=ask_headers, json=ask_data)

    print(response.status_code)
    print(response.text)

ask_ki()

ReadTimeout: timed out